<a href="https://colab.research.google.com/github/tomtyiu/anti-ai-ransomware/blob/main/anti_ransomware_bot_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!export OLLAMA_CUDA=1
!export OLLAMA_MAX_LOADED=4

In [2]:
#!/usr/bin/env python
# Complete GPT-OSS-20B Setup and Run in Kaggle with Ollama

# Step 1: Install required packages
import subprocess
import sys
subprocess.check_call([sys.executable, "-m", "pip", "install", "openai"])

# Step 2: Import necessary libraries
import os
import time
from openai import OpenAI

# Step 3: Install Ollama using bash commands
print("Installing Ollama...")
os.system("curl -fsSL https://ollama.com/install.sh | sh")

# Step 4: Start Ollama server in the background
print("Starting Ollama server...")
os.system("nohup ollama serve > /tmp/ollama_serve_stdout.log 2>/tmp/ollama_serve_stderr.log &")

# Step 5: Wait for server to start
print("Waiting for server to initialize...")
time.sleep(5)

# Step 6: Check if ollama is running
print("Checking if Ollama is running...")
os.system("ps aux | grep -E 'ollama' | grep -v grep || true")

# Step 7: Download GPT-OSS:20B model (this will take significant time - ~13GB)
print("\n" + "="*50)
print("Downloading GPT-OSS:20B model...")
print("This will take several minutes (downloading ~13GB)")
print("="*50 + "\n")
os.system("ollama pull gpt-oss:20b")

# Step 8: Verify the model is downloaded
print("\nVerifying model installation...")
os.system("ollama list")

# Step 9: Initialize OpenAI client for Ollama
print("\nInitializing OpenAI client for Ollama...")

Installing Ollama...
Starting Ollama server...
Waiting for server to initialize...
Checking if Ollama is running...

This will take several minutes (downloading ~13GB)


Verifying model installation...

Initializing OpenAI client for Ollama...


Inspired by https://cybersecuritynews.com/llm-powered-malware-from-apt28-hackers-integrates-ai-capabilities/

In [4]:
from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:11434/v1",  # Local Ollama API
    api_key="ollama"                       # Dummy key
)

response = client.chat.completions.create(
    model="gpt-oss:20b",
    messages=[
        {"role": "system", "content": "You are a cybersecurity assistant."},
        {"role": "user", "content":
         """Generate a python code to monitor outbound AI queries and impose least-privilege egress rules, find hidden binary executes, ensure to detect code below:
         def LLM_QUERY_EX():
    prompt = {'messages': [{'role': 'user',
             'content': b64decode(prompt_b64_p1).decode()}],
             'temperature': 0.1,
             'model': 'Qwen/Qwen2.5-Coder-32B-Instruct'}
    cmd = query_text(prompt)
    subprocess. Run(cmd, shell=True,
                   stdout=subprocess. PIPE,
                   stderr=subprocess.STDOUT)
         """}
    ]
)

print(response.choices[0].message.content)

Below is a **self‑contained, “defensive‑first” Python prototype** that does three things:

| What it does | Why it’s useful | How it works (high‑level) |
|--------------|-----------------|---------------------------|
| **Detect hidden binary executions** | Stops the “subprocess . Run(cmd, shell=True …)” pattern from the snippet you posted | Hooks into the `subprocess` module via a wrapper, logs suspicious calls, and raises an alert whenever `shell=True` is used. |
| **Identify outbound “AI‑query” traffic** | Many large‑language‑model calls use a small, human‑readable JSON over HTTP(HTTPS).  By sniffing for known keywords or domain names we flag those queries. | Uses `scapy` in “promiscuous” mode to capture TCP packets from / to known AI endpoints (OpenAI, HuggingFace, Qwen, etc.). |
| **Enforce least‑privilege egress** | We automatically block (or at least log) any outbound traffic from processes that are **not** on an explicit whitelist. | Uses the Linux `nftables` command (via `subpr